<a href="https://colab.research.google.com/github/hws2002/MachineLearning_PytorchNScikitLearn/blob/master/chapter6/Chaper6_2_k_fold_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.2 k-겹 교차 검증을 사용한 모델 성능 평가
이 절에서는 보편적인 교차 검증 기법인 홀드아웃 방법과 k-겹 교차 검증을 배워보자.  
이런 방법들은 모델의 일반화 성능, 즉 처음 본 데이터에 모델이 얼마나 잘 작동하는지 신뢰할 만한 추정을 하도록 도와줌

## 6.2.1 홀드아웃 방법

## 6.2.2 k-겹 교차 검증
k-겹 교차 검증에서는 중복을 허용하지 않고 훈련 데이터셋을 k개의 폴드(fold)로 랜덤하게 나눔  

k-1개의 폴드( 이를 훈련 폴드라고 함) 로 모델을 훈련하고 나머지 하나의 폴드( 이를 테스트 폴드라고 함) 로 성능을 평가.  

그 다음 서로 다른 독립적인 폴드에서 얻은 성능 추정을 기반으로 모델의 평균 성능을 계산.  

홀드아웃 방법에 비해 훈련 데이터셋의 분할에 덜 민감한 성능 추정을 얻을 수 있음.  

일반적으로 모델 튜닝에 k-겹 교차 검증을 사용함.( 즉, 테스트 데이터셋에서 모델의 성능을 평가할 때 만족할 만한 일반화 성능을 내는 최적의 하이퍼파라미터 값을 찾기 위해 사용)

In [9]:
import pandas as pd
df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data',
    header = None
)
df.columns = [
    'ID',
    'Diagnosis',
    'radius1', 'texture1', 'perimeter1', 'area1', 'smoothness1', 'compactness1', 'concavity1', 'concave_points1', 'symmetry1', 'fractal_dimension1',
    'radius2', 'texture2', 'perimeter2', 'area2', 'smoothness2', 'compactness2', 'concavity2', 'concave_points2', 'symmetry2', 'fractal_dimension2',
    'radius3', 'texture3', 'perimeter3', 'area3', 'smoothness3', 'compactness3', 'concavity3', 'concave_points3', 'symmetry3', 'fractal_dimension3'
]
X, y = df.iloc[:,2:].values , df.iloc[:,1].values


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(le.classes_)
print(y)

['B' 'M']
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 0 0
 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1
 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 1 0 0
 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 1 

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, stratify = y, random_state = 1)
y_train_bc = np.bincount(y_train)
print(y_train_bc)
print(f'ratio : {y_train_bc[0]/ y_train_bc.sum() :.3f} , {y_train_bc[1] / y_train_bc.sum() :.3f}')

[285 170]
ratio : 0.626 , 0.374


In [17]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train_std = sc.fit_transform(X_train)
# X_test_std = sc.transform(X_test)
from sklearn.decomposition import PCA
# pca = PCA(n_components = 2)
# X_train_pca = pca.fit_transform(X_train_std)
# X_test_std = pca.transform(X_test_std)

from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(multi_class = 'ovr',
                        #  random_state = 1,
                        #  solver = 'lbfgs')
pipe_lr = make_pipeline(StandardScaler(),
                        PCA(n_components=2),
                        LogisticRegression(random_state = 1))

In [21]:
from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(n_splits = 10).split(X_train,y_train)
scores = []
for k, (train,test) in enumerate(kfold):
  pipe_lr.fit(X_train[train], y_train[train])
  score = pipe_lr.score( X_train[test], y_train[test])
  scores.append(score)
  y_train_bc = np.bincount(y_train[train])
  print(f'폴드 : {k+1:2d}, '
        f'클래스 분포 : {np.bincount(y_train[train])}, '
        f'클래스 비율 : ({y_train_bc[0]/ y_train_bc.sum() :.3f} , {y_train_bc[1] / y_train_bc.sum() :.3f}), '
        f'정확도: {score:.3f}')


폴드 :  1, 클래스 분포 : [256 153], 클래스 비율 : (0.626 , 0.374), 정확도: 0.935
폴드 :  2, 클래스 분포 : [256 153], 클래스 비율 : (0.626 , 0.374), 정확도: 0.935
폴드 :  3, 클래스 분포 : [256 153], 클래스 비율 : (0.626 , 0.374), 정확도: 0.957
폴드 :  4, 클래스 분포 : [256 153], 클래스 비율 : (0.626 , 0.374), 정확도: 0.957
폴드 :  5, 클래스 분포 : [256 153], 클래스 비율 : (0.626 , 0.374), 정확도: 0.935
폴드 :  6, 클래스 분포 : [257 153], 클래스 비율 : (0.627 , 0.373), 정확도: 0.956
폴드 :  7, 클래스 분포 : [257 153], 클래스 비율 : (0.627 , 0.373), 정확도: 0.978
폴드 :  8, 클래스 분포 : [257 153], 클래스 비율 : (0.627 , 0.373), 정확도: 0.933
폴드 :  9, 클래스 분포 : [257 153], 클래스 비율 : (0.627 , 0.373), 정확도: 0.956
폴드 : 10, 클래스 분포 : [257 153], 클래스 비율 : (0.627 , 0.373), 정확도: 0.956


In [23]:
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV 정확도 : {mean_acc:.3f} +/- {std_acc:.3f}')


CV 정확도 : 0.950 +/- 0.014


In [24]:
# 사이킷런의 k-겹 교차 검증 함수를 활용해도 됨
from sklearn.model_selection import cross_val_score
scores = cross_val_score(estimator = pipe_lr,
                         X = X_train,
                         y = y_train,
                         cv = 10,
                         n_jobs = 1 # 사용할 CPU 코어 개수
                        )
print(f'CV 정확도 점수 : {scores}')

print(f'CV 정확도 : {np.mean(scores):.3f}'
      f'+/- {np.std(scores):.3f}')

CV 정확도 점수 : [0.93478261 0.93478261 0.95652174 0.95652174 0.93478261 0.95555556
 0.97777778 0.93333333 0.95555556 0.95555556]
CV 정확도 : 0.950+/- 0.014
